# 0. Import required packages.

In [6]:
import copy
import time

# 1. Define a `Game` Class for containing the Tic-Tac-Toe game.

In [10]:
class Game(object):
    """A tic-tac-toe game."""

    def __init__(self, grid):
        """Instances differ by their grid marks."""
        self.grid = copy.deepcopy(grid) # No aliasing!

    def display(self):
        """Print the game board."""
        for row in self.grid:
            print(row)

    def moves(self):
        """Return a list of possible moves given the current marks.
        
        Parameters
        ----------
            None
        
        Returns
        -------
            possible_moves: List of Tuples containing (row, column) of available spaces on the grid.
        
        Author: Miguel Agueda-Cabral
        """
        
        possible_moves = []
        for row_index, row in enumerate(self.grid):
            for column_index, value in enumerate(row):
                if value == '-':
                    position = (row_index, column_index)
                    possible_moves.append(position)
        
        return possible_moves

    def neighbor(self, move, mark):
        """Return a Game instance like this one but with one move made.
        
        Parameters
        ----------
            move: Tuple containing (row, column) indices of move to be made.
            mark: String identity of player ('X', 'O'). 
        
        Returns
        -------
            neighbor_game: A copy of `self` with `move` applied.
        
        Author: Miguel Agueda-Cabral
        """
        
        neighbor_game = Game(self.grid)
        (row_idx, col_idx) = move
        neighbor_game.grid[row_idx][col_idx] = mark
        
        return neighbor_game

    def utility(self): # author Walter Jordan
        """Return the minimax utility value of this game:
        1 = X win, -1 = O win, 0 = tie, None = not over yet."""
        # need to check for positions, I should just define it statically even if that is inefficient
        winpos = [[(0,0),(0,1),(0,2)], [(1,3),(1,4),(1,5)], [(2,6),(2,7),(2,8)], 
                  [(0,0),(1,3),(2,6)], [(0,1),(1,4),(2,7)], [(0,2),(1,5),(2,8)], 
                  [(0,0),(1,4),(2,8)], [(0,2),(1,4),(2,6)]]
        
        for x in winpos:
            if self.grid[x[0[0]]][x[0[1]]] == self.grid[x[1[0]]][x[1[1]]] == self.grid[x[2[0]]][x[2[1]]]:
                if self.grid[x[0]] == 'X':
                    return 1
                if self.grid[x[0]] == 'O':
                    return -1
                #if self.grid[x] == '-': # probably need to fix this
                    #return 0
        if '-' not in self.grid:
            return 0
        return None

<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:66: SyntaxWarning: 'int' object is not subscriptable; perhaps 

   # 2. Define an `Agent` Class which can manipulate and solve an instance of `Game`.

In [8]:
class Agent(object):
    """Knows how to play tic-tac-toe."""

    def __init__(self, mark):
        """Agents use either X or O."""
        self.mark = mark
        
    def maxvalue(self, game, opponent):
        """Compute the highest utility this game can have."""
        
        if game.utility() != 0:
            return game.utility()
        
        v = -np.inf
        poss_actions = game.moves()
        for a in poss_actions:
            v = max(v, self.minvalue(game, opponent))
        
        return v

    def minvalue(self, game, opponent): # author Walter Jordan
        """Compute the lowest utility this game can have."""
        possibles = game.moves()
        bestlocation = None
        bestval = None
        for x in possibles:
            game.neighbor(x, opponent.mark)
            util = game.utility()
            if util is not None:
                val = util
            else:
                val, mv = self.maxvalue(game, opponent)
            if val < bestval or bestval == None:
                bestval = val
                bestlocation = x
        return bestval, bestlocation
            



In [9]:
def main():
    """Create a game and have two agents play it."""

    game = Game([['-','-','-'], ['-','-','-'], ['-','-','-']])
    game.display()

    maxplayer = Agent('X')
    minplayer = Agent('O')

    while True:

        (value, move) = maxplayer.maxvalue(game, minplayer)
        game = game.neighbor(move, maxplayer.mark)
        time.sleep(1)
        game.display()
        
        if game.utility() is not None:
            break
        
        (value, move) = minplayer.minvalue(game, maxplayer)
        game = game.neighbor(move, minplayer.mark)
        time.sleep(1)
        game.display()
        
        if game.utility() is not None:
            break

if __name__ == '__main__':
    main()

['-', '-', '-']
['-', '-', '-']
['-', '-', '-']


TypeError: list indices must be integers or slices, not tuple